<a href="https://colab.research.google.com/github/stogaja/NLP-PROJECT/blob/main/Semantic_text_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.0 MB/s 
     |████████████████████████████████| 120 kB 58.9 MB/s 
     |████████████████████████████████| 115 kB 70.0 MB/s 
     |████████████████████████████████| 212 kB 70.1 MB/s 
     |████████████████████████████████| 127 kB 69.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 4.9 MB 25.6 MB/s 
     |████████████████████████████████| 1.3 MB 50.2 MB/s 
     |████████████████████████████████| 6.6 MB 45.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a7f5dd20

In [2]:
#loading training set
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from datasets import load_dataset


# Load the English STSB dataset
stsb_dataset = load_dataset('stsb_multi_mt', 'en')
stsb_train = pd.DataFrame(stsb_dataset['train'])
stsb_test = pd.DataFrame(stsb_dataset['test'])

# Check loaded data
print(stsb_train.shape, stsb_test.shape)
stsb_test.head()

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

(5749, 3) (1379, 3)


,sentence1,sentence2,similarity_score
0,A girl is styling her hair.,A girl is brushing her hair.,2.5
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,3.6
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,5.0
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,4.2
4,A man is playing a harp.,A man is playing a keyboard.,1.5


## Creating helper functions
* The first function is to pre-process texts by lemmatizing, lowercasing, and removing numbers and stop words.
* The second function takes in two columns of text embeddings and returns the row-wise cosine similarity between the two columns.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load("en_core_web_sm")

def text_processing(sentence):
    """
    Lemmatize, lowercase, remove numbers and stop words
    
    Args:
      sentence: The sentence we want to process.
    
    Returns:
      A list of processed words
    """
    sentence = [token.lemma_.lower()
                for token in nlp(sentence) 
                if token.is_alpha and not token.is_stop]
    
    return sentence


def cos_sim(sentence1_emb, sentence2_emb):
    """
    Cosine similarity between two columns of sentence embeddings
    
    Args:
      sentence1_emb: sentence1 embedding column
      sentence2_emb: sentence2 embedding column
    
    Returns:
      The row-wise cosine similarity between the two columns.
      For instance is sentence1_emb=[a,b,c] and sentence2_emb=[x,y,z]
      Then the result is [cosine_similarity(a,x), cosine_similarity(b,y), cosine_similarity(c,z)]
    """
    cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
    return np.diag(cos_sim)

In [5]:
data = (pd.read_csv("/content/SBERT_data.csv")).drop(['Unnamed: 0'], axis = 1)

prompt = input("Enter prompt: ")
data['prompt']= prompt
data.rename(columns = {'target_text':'sentence2', 'prompt':'sentence1'}, inplace = True)
data['sentence2'] = data['sentence2'].astype('str')
data['sentence1']  = data['sentence1'].astype('str')

data.head()

Enter prompt: jsdwwhw


,input_text,sentence2,sentence1
0,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,When true indicates the purpose of the extensi...,jsdwwhw
1,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the creditor organization type exemp...,jsdwwhw
2,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,A free-form text field used to collect additio...,jsdwwhw
3,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,Specifies the Loan Program that exempts the tr...,jsdwwhw
4,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,A free-form text field used to collect additio...,jsdwwhw


In [6]:
from sentence_transformers import CrossEncoder
XpathFinder = CrossEncoder("cross-encoder/stsb-roberta-base")
sentence_pairs = []
for sentence1, sentence2 in zip(data['sentence1'],data['sentence2']):
  sentence_pairs.append([sentence1, sentence2])

data['SBERT CrossEncoder_Score'] = XpathFinder.predict(sentence_pairs, show_progress_bar = True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/142 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Batches:   0%|          | 0/675 [00:00<?, ?it/s]

In [7]:
data.sort_values(by=['SBERT CrossEncoder_Score'], ascending=False)


,input_text,sentence2,sentence1,SBERT CrossEncoder_Score
11735,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,A description of a charge an adjustment a loss...,jsdwwhw,0.755061
11736,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,A description of a charge an adjustment a loss...,jsdwwhw,0.739398
11725,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,The monetary amount of a charge an adjustment ...,jsdwwhw,0.731553
9829,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/LOANS/L...,The number of hours by which the executed loan...,jsdwwhw,0.715355
21158,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/ASSETS/...,An unparsed legal description of a parcel of r...,jsdwwhw,0.691429
...,...,...,...,...
19932,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/PARTIES...,E W N S NE NW SE SW which belongs to an addre...,jsdwwhw,0.014708
19868,DOCUMENT/DEAL_SETS/DEAL_SET/DEALS/DEAL/PARTIES...,E W N S NE NW SE SW which belongs to an addre...,jsdwwhw,0.014708
19934,DOCUMENT/PARTIES/PARTY/ADDRESSES/ADDRESS/Stree...,E W N S NE NW SE SW which belongs to an addre...,jsdwwhw,0.009638
19870,DOCUMENT/PARTIES/PARTY/ADDRESSES/ADDRESS/Stree...,E W N S NE NW SE SW which belongs to an addre...,jsdwwhw,0.009638


In [9]:
import pickle

filename = 'XpathFinder1.sav'
pickle.dump(XpathFinder, open(filename, 'wb'))

In [10]:
!pip install -q streamlit

     |████████████████████████████████| 9.1 MB 5.1 MB/s 
     |████████████████████████████████| 181 kB 71.2 MB/s 
     |████████████████████████████████| 235 kB 70.5 MB/s 
     |████████████████████████████████| 4.7 MB 44.9 MB/s 
     |████████████████████████████████| 164 kB 74.9 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 51 kB 6.9 MB/s 


In [15]:
%%writefile app.py
import io
import netrc
import pickle
import sys
import pandas as pd
import streamlit as st
# let's import sentence transformer
import sentence_transformers
import torch
#######################################

st.markdown(
    f"""
<style>
    .reportview-container .main .block-container{{
        max-width: 90%;
        padding-top: 5rem;
        padding-right: 5rem;
        padding-left: 5rem;
        padding-bottom: 5rem;
    }}
    img{{
    	max-width:40%;
    	margin-bottom:40px;
    }}
</style>
""",
    unsafe_allow_html=True,
)

# # let's load the saved model
loaded_model = pickle.load(open('XpathFinder1.sav', 'rb'))
#loaded_model = pickle.load('XpathFinder1.sav', map_location='cpu')


#class CPU_Unpickler(pickle.Unpickler):
#    def find_class(self, module, name):
#        if module == 'torch.storage' and name == '_load_from_bytes':
#            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
#        else:
#            return super().find_class(module, name)
#

#loaded_model = CPU_Unpickler(open('XpathFinder1.sav', 'rb')).load()


# here is how to create containers
header_container = st.container()
stats_container = st.container()

# the header
with header_container:

    # different levels of text you can include in your app
    st.title("Semantic Text Similarity App")
    st.header("Hello!, Let's match some words!!!")

# Another container
with stats_container:
    # collect input using free text
    text_input = st.text_input("Enter the text in the space below ...")

    # let's pass the input to the loaded_model with torch compiled with cuda
    if text_input:
        # let's get the result
        result = loaded_model.predict([text_input])
        # let's show the result
        st.write(result)

Writing app.py


In [21]:
!pip install pyngrok 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from pyngrok import ngrok

In [23]:
ngrok.set_auth_token("2EolZMbfSIIvEtovVRdA5Gi7KJY_3AKjE4jHU5udEjXfT6noE")

In [25]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'


PyngrokNgrokHTTPError: ignored